# Worksheet 01: Introduction to Generative Modelling

## Learning Objectives

After completing this week's lecture and tutorial work, you will be able to:

1. Give an example of a question that could be answered by generative modelling.
2. Explain how linear regression can be used to approximate the underlying mechanism that generated the data (quantitative response and input variables).
3. Interpret the estimated coefficients and 𝑝𝑝-values derived from theoretical results for a simple linear regression (i.e., one input variable).
4. Discuss the assumptions made to estimate the simple linear regression coefficients and approximate their sampling distribution.
5. Explain how to approximate the sampling distribution of the simple linear regression coefficient estimators using bootstrapping.
6. Contrast the sampling distribution approximated using theoretical results with bootstrapping alternatives for a simple linear regression setting.
7. Compute confidence intervals for the simple linear regression coefficients using theoretical approximations and bootstrapping results.
8. Write a computer script to perform a simple linear regression analysis.


In [ ]:
# Run this cell before continuing.
library(tidyverse)
library(infer)
library(cowplot)
library(broom)
source("tests_worksheet_01.R")

## Introduction: Generative models

As data scientists, we are often interested in understanding the relationship between variables in our data using *models*. For example:

- which variables are associated with a response of interest? 

- can we model the relationship between the response and the input variables? Is a linear model adequate?

- which variables are positively/negatively associated with the response? 

- does the relationship between the response and an input variable depend on the values of the other variables?

Linear Regression Models can be used to answer these questions, providing a unifying framework to study the relation between different types of variables and a continuous response. Research in linear models has been focused on three important aspects: *estimation, inference*, and *prediction*. 

- **Estimation**: how to estimate the true (but unknown) relation between the response and the input variables

- **Inference**: how to use the model to infer information about the unknown relation between variables

- **Prediction**: how to use the model to predict the value of the response for new observations 

These three aspects are intimately connected, and understanding their relationships is key to unlocking the full potential of Linear Regression Models.

The purpose of this worksheet is to practice building, interpreting and testing Simple Linear Regression (SLR) models (note that "*Simple*" refers to a linear model with only *one* input variable - it is not because it is easy!) . You will also become familiar with `R` functions such as `lm()` and `broom()` to accomplish these tasks. 

## Case study: Cancer Mortality

In this worksheet, you will use the dataset `US_cancer_data`, which contains data on cancer mortality rates and different demographic and medical variables in American counties. 

**The question:** What is the relationship between cancer mortality rates and different demographic and medical variables?

The data come from [data.world](https://data.world/nrippner/ols-regression-challenge) and other sources: [census.gov](http://census.gov/), [clinicaltrials.gov](http://clinicaltrials.gov/), and [cancer.gov](http://cancer.gov/). All values were collected in the 2010s but varied according to source. (Warning: Recall the importance of using a *random* sample to obtain representative summaries and broad conclusions!)

In the [source documentation](https://data.world/nrippner/ols-regression-challenge) you can find the definition of each of the selected variables:

- `TARGET_deathRate`: a continuous variable that measures cancer mortality per capita (for every 100,000 inhabitants), obtained as an average of data collected from 2010 to 2016.

- `povertyPercent`: a continuous variable that measures the percentage of the county's population in poverty, based on 2013 American Census estimates.

- `PctPrivateCoverage`: a continuous variable that measures the percentage of county residents with private health coverage, based on 2013 American Census estimates. 

Let's start by reading this dataset!

In [ ]:
# Run this cell to read the dataset

US_cancer_data <- 
    read_csv("data/US_county_cancer_data.csv") %>%
    select(TARGET_deathRate, povertyPercent, PctPrivateCoverage)

## PART I: Estimation

### 1.  The Model

You can hypothesize that each county's cancer mortality is linearly related to the average level of poverty. In this worksheet, you will quantitatively study the association, at the county level, between `TARGET_deathRate` and `povertyPercent` using an SLR. 

**Question 1.0**
<br>{points: 1}

Within the context of this case study, answer the following:

**1.0.0.** Which variable will you choose as the response variable? Answer with the column's name from `US_cancer_data`.

**1.0.1.** Which variable will you choose as the input variable? Answer with the column's name from `US_cancer_data`.


*Assign your answers to the objects `answer1.0.0` (character type surrounded by quotes) and `answer1.0.1` (character type surrounded by quotes).*

In [ ]:
# (answer1.0.0 <- ...)
# (answer1.0.1 <- ...)

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_1.0()

**Question 1.1**
<br>{points: 1}

Using `US_cancer_data`, create a scatterplot of the response variable (in the y-axis) versus the input variable (in the x-axis). Call the resulting object `cancer_poverty_scatterplot` (note: it is important to display units of variables in plots to allow their proper interpretation!)

*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 7) # Adjust these numbers so the plot looks good in your desktop.

# cancer_poverty_scatterplot <- ggplot(..., aes(..., ...)) +
#   ...() +
#   xlab(...) +
#   ylab(...) +
#   theme(
#     text = element_text(size = 16.5),
#     plot.title = element_text(face = "bold"),
#     axis.title = element_text(face = "bold")
#   )
# cancer_poverty_scatterplot

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_1.1()

**Question 1.2**
<br>{points: 1}

Based on the `cancer_poverty_scatterplot`, how would you describe the association between the response and the input variables?

A. Negative.

B. Positive.

C. No association.

*Assign your answer to an object called `answer1.2`. Your answer should be one of `"A"`, `"B"` or `"C"` surrounded by quotes.*

In [ ]:
# answer1.2 <- ...

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_1.2()

### 2. Estimation of the regression line

Let's start by formally defining the linear model. Let ${(X_i,Y_i), i = 1, \ldots , n,}$ be a <font color="red">random sample</font> of size $n$ from the population. 

In this example:

- $Y_i$: the cancer mortality per capita for the $i$th county

- $X_i$: percentage of the populace in poverty for the $i$th county

- the index $i$ identifies the county and goes from $1$ to $n$.

The model equation is:

$$
Y_i = \beta_0 + \beta_1 X_i + \epsilon_i
$$
where $\epsilon_i$ is the error term. Observations from a random sample won't be perfectly lined. The error term contains all factors that deviate $Y_i$ from that assumption.

In Statistics, $Y$ is usually used to name the response variable and $X$ the covariates. This is useful to study theoretical properties or when the model has too many variables.

However, we can also use more meaningful names for the variables

$\quad\quad\quad\quad\quad\quad\quad\quad\quad$ `TARGET_deathRate` = $\beta_0$ + $\beta_1\times$`povertyPercent` + $\epsilon_i$

Note that this model is an approximation (actually, pretty much <u>all models are approximations</u>) and cannot be considered exact. For example, the equation above states that we expect the cancer mortality rate to change at a constant rate based on the percentage of the population living in poverty. Is this a reasonable assumption?

**The best line:**

But what values should we use for $\beta_0$ and $\beta_1$? These are our population parameters. As we know, population parameters are *unknown* and *non-random*. We need to estimate them from the data (a random sample)! But how?

There are infinite lines to choose from. Which one is the best line?

In [ ]:
US_cancer_data %>% ggplot(aes(povertyPercent, TARGET_deathRate)) + theme(axis.text.x = element_text(angle = 90))+
    geom_point()+ 
    geom_abline(intercept=145,slope=1.5, size=2, col = "blue")+
    geom_abline(intercept=200,slope=-1, size=2, col = "orange")+
    geom_abline(intercept=100,slope=5, size=2, col = "red")+
    geom_abline(intercept=155,slope=1.2, size=2, col = "green")+
    geom_abline(intercept=147,slope=1.9, size=2, col = "purple")+
    xlab("Populace in Poverty (%)") +
    ylab("Cancer Mortality per Capita (cases/100,000)")

**Question 2.0**
<br>{points: 1}

How would you choose the *best* line? 

**A.** The line that contains most data points 

**B.** The line that minimizes the distance of the points to the line 

**C.** The line that looks the best upon visual inspection

*Assign your answer to an object called `answer2.0`. Your answer should be one of `"A"`, `"B"`, or `"C"` surrounded by quotes.*

In [ ]:
# answer2.0 <- ...

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_2.0()

**Question 2.1**
<br>{points: 1}

In order to determine the "best" line, we must first establish what we mean by "best". One way to do this is to find the line "closer" to all the points. However, to measure the distance between the points and the line, we need to determine a suitable criterion. Which of the following three criteria would you select to define the "distance of a point to the line"? (<u>Hint:</u> think about the variable we are trying to explain.)


![](img/dist.png)

Figure by Prof. Joel Ostblom

*Assign your answer to an object called `answer2.1`. Your answer should be one of `"A"`, `"B"`, or `"C"` surrounded by quotes.*

In [ ]:
# answer2.1 <- ...

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_2.1()



Many methods can be used to estimate the true regression line depending on the criteria used to define "optimal"! **Least Squares (LS) method** minimizes the sum of the *squares of the residuals*! The residual of an observation is the difference between its response value and its predicted response on the line (represented by the dotted vertical line in Item B above). Check [this application](http://setosa.io/ev/ordinary-least-squares-regression/). 

> **Activity**: discuss why the *residual* is different from the *error term*?






### 3. Least Square (LS) in R

We will use the `lm` function in R to obtain the Least Square (LS) Estimates using the data in `US_cancer_data`. The LS method is not the only method to estimate the regression coefficients. However, it is the default method in `lm`.

The `lm` function has two main arguments:

- `formula`: takes the form `response ~ input`.
- `data`: takes a data frame in a tidy format.

In the `lm` function, there are some helpers to handle the inputs. We'll explore them further as the course progresses, but for now, let's start exploring two: 
 - If you use `lm(response ~ ., data = df)`, all variables in the dataset `df`, except the `response`, will be used as predictors.
 - `lm(response ~ input - 1, data = df)` forces the estimated intercept to be 0. Never do this unless you know what you are doing and why. 

In this dataset we have sample of size in $n = 3047$ to estimate the regression coefficients. To examine the properties of the estimator let's start with a smaller sample and see what happens as the sample size increases.

**Question 3.0**
<br>{points: 1}

From the pool of American counties `US_cancer_data`, use the function `slice_sample()` to collect one random sample of size `250` and call it `US_cancer_sample250`.


*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
set.seed(123) # DO NOT CHANGE!

# US_cancer_sample250 <- 
#    US_cancer_data %>%
#    ...(n = )

# your code here
fail() # No Answer - remove if you provide an answer

head(US_cancer_sample250)

In [ ]:
test_3.0()

**Question 3.1**
<br>{points: 1}

It is time to use `R` to estimate the Simple Linear Regression (SLR) using `US_cancer_sample250`. This estimated model can be used to evaluate if there exists a linear association between cancer mortality and poverty.

Use the `lm()` function to estimate the SLR. 

Store this estimated model in the variable `SLR_cancer_sample250`.

*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
# SLR_cancer_sample250 <- ...(formula = , data = ...)
# SLR_cancer_sample250

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_3.1()

**Question 3.2: Interpretation of the estimated line**
<br>{points: 1}

What is the correct interpretation of the regression estimated slope in `SLR_cancer_sample250_results`?

A. The effect of a one percent increase of the county's populace in poverty is 1.52 increase in the cancer mortality per capita (cases/100,000).

B. One percent increase of the county's populace in poverty causes 1.52 increase in cancer mortality per capita (cases/100,000).

C. A one percent increase of the county's populace in poverty is associated with a 1.52 increase in cancer mortality per capita (cases/100,000).

*Assign your answer to an object called `answer3.2`. Your answer should be one of `"A"`, `"B"`, or `"C"` surrounded by quotes.*

In [ ]:
# answer3.2 <- ...

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_3.2()

**Note**: since this dataset comes from an observational study, an SLR can be used to describe association, but not to establish a causal relationship between the covariate and the response.

**Question 3.3: Visualization of the estimated line**
<br>{points: 1}

Using `US_cancer_sample250`, create a scatterplot of`TARGET_deathRate` versus `povertyPercent` and add the estimated SLR. The `ggplot()` object's name will be `SLR_cancer_sample250_plot`. Take a look at `geom_smooth`. 

*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
# SLR_cancer_sample250_plot <- 
#     US_cancer_sample250 %>%
#     ggplot(aes(x = ..., y = ...)) +
#     ...() +
#     ...(method = ..., se = FALSE, size = 1.5) +
#     coord_cartesian(xlim = c(0, 50), ylim = c(50, 400)) +
#     xlab(...) +
#     ylab(...) +
#     ggtitle(...) +
#     theme(
#         text = element_text(size = 16.5),
#         plot.title = element_text(face = "bold"),
#         axis.title = element_text(face = "bold"))


# your code here
fail() # No Answer - remove if you provide an answer

SLR_cancer_sample250_plot 


In [ ]:
test_3.3()

**Question 3.4**
<br>{points: 1}

Considering the SLR model estimated in this exercise, which of the following questions relates to inference and estimation, and which relates to prediction? 

| **Question** | **Type** |
| ------------------------------- | ----------------------- |
| How can we determine an association between the expected cancer mortality per capita (cases/100,000) and the county’s populace living in poverty of all American counties? | `answer3.4.0` |
| We observe a new American county with 14% of its populace living in poverty. What cancer mortality per capita (cases/100,000) should we expect? | `answer3.4.1` |

The right column of the table is empty but should describe one of the following: 

**A.** Prediction.

**B.** Inference and estimation.

*Assign your answers to the objects `answer3.4.0` and `answer3.4.1`. Your answer should each be a single character (`"A"` or `"B"`) surrounded by quotes.*

In [ ]:
# answer3.4.0 <- ...
# answer3.4.1 <- ...

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_3.4()

## PART II: Inference

Since the parameters of the linear model ($\beta_0$ and $\beta_1$) are unknown, you use a random sample to estimate with a LS estimator ($\hat{\beta}_0$ and $\hat{\beta}_1$). 

In particular, in this case study, the estimated intercept and slope in `SLR_cancer_sample250_results` are computed using the $n = 250$ sampled observations from `US_cancer_data`. We can think that the *estimates* are "good" guesses about the population parameters based on our data. 

In this section, we will examine how we can infer information about the *population*'s parameters from the *estimated* parameters. 

### A note on randomness of the estimators

It's important to understand that $\hat{\beta}_0$ and $\hat{\beta}_1$ are estimates computed from random samples and, therefore, are themselves random variables. In other words, the values of $\hat{\beta}_0$ and $\hat{\beta}_1$ will vary depending on the particular sample used to compute them. This means that different samples of the same population will yield different estimates of $\hat{\beta}_0$ and $\hat{\beta}_1$.

Let's see an example! Recall the estimates we obtained with the sample taken:

In [ ]:
SLR_sample250_0 <- 
    tidy(lm(formula = TARGET_deathRate ~ povertyPercent, data = US_cancer_sample250)) %>% 
    select(estimate)

many_SLR <- SLR_sample250_0 
many_SLR

Take another sample from the full dataset and estimate the regression line. A couple of notes that are important to keep in mind:

- This is a pedagogical exercise -- in practice, we rarely take multiple samples!
- This is NOT bootstrapping! Why?


In [ ]:
set.seed(301)

US_cancer_sample250_1 <- rep_sample_n(US_cancer_data, size = 250)

In [ ]:
# NEW POINT ESTIMATES

SLR_sample250_1 <- 
    tidy(lm(formula = TARGET_deathRate ~ povertyPercent, data = US_cancer_sample250_1))  %>% 
    select(estimate)

many_SLR <- many_SLR %>% bind_cols(SLR_sample250_1)
many_SLR

Let's do this again and take yet another sample. 

In [ ]:
set.seed(30)

US_cancer_sample250_2 <- rep_sample_n(US_cancer_data, size = 250)

In [ ]:
# ANOTHER POINT ESTIMATES

SLR_sample250_2 <- 
    tidy(lm(formula = TARGET_deathRate ~ povertyPercent, data = US_cancer_sample250_2))  %>% 
    select(estimate)

many_SLR <- many_SLR  %>% bind_cols(SLR_sample250_2)
many_SLR

As we collect new samples, we observe that the regression parameters generate varying estimates. This is a common occurrence, as you learned in STAT 201, where you were introduced to the idea that estimators have sampling distributions. In other words, the estimates of regression parameters vary from one sample to the next, reflecting the natural variability that is inherent in statistical inference, and they also have sampling distributions.

### 4. The standard error

The variation of these estimates from sample to sample is measured by their standard deviation, which has a special name: *the standard error* (SE). But in practice, how can we compute the standard error if we have only 1 sample?

We have different ways of answering this question:

1. use a theoretical result, which is what the `lm` function does;

2. use bootstrapping (similarly to what you did in STAT 201);

**Question 4.0**
<br>{points: 1}

Use the `broom` package's `tidy()` to obtain the estimated coefficients, associated standard errors, $t$-statistics, and $p$-values obtained from the random sample `SLR_cancer_sample250`. 

Store them in the variable `SLR_cancer_sample250_results` whose columns are the following:

- The first column has the names of the regression terms.

- The second column shows the values of the estimated coefficients of the regression line, $\hat{\beta}_0$ and $\hat{\beta}_1$ 

- The remaining three columns have important quantities to assess uncertainty and test hypotheses about the regression terms (we'll learn more about these quantities soon).

*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
# SLR_cancer_sample250_results <- 
#    ...(SLR_cancer_sample250) %>% 
#    mutate_if(is.numeric, round, 2) # This just rounds the numbers to two decimal places 

# your code here
fail() # No Answer - remove if you provide an answer

SLR_cancer_sample250_results

In [ ]:
test_4.0()

### 5. Hypothesis Tests

The object `SLR_cancer_sample250_results` contains useful information to test some hypotheses about the regression coefficients.

### 5.1 The null hypothesis

In other hypothesis tests, the null hypothesis claims a *null* state, i.e., the status-quo. In SLR, the status-quo is the absence of a relation between the response and the input variable(s). 

**Question 5.1**
<br>{points: 1}

Suppose we want to test if the there exist a linear association between cancer mortality and poverty. Which of the following null hypotheses is correct:

**A.** $H_0: \hat{\beta}_1 = 0 $

**B.** $H_0: \hat{\beta}_0 = 0 $

**C.** $H_0: \beta_1 = 0$ 

**D.** $H_0: \beta_0 = 0$ 

*Assign your answer to an object called `answer5.1`. Your answer should be one of `"A"`, `"B"`, `"C"`, or `"D"` surrounded by quotes.*

In [ ]:
# answer5.1 <- 

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_5.1()

### 5.2 The alternative hypothesis 

The alternative hypothesis reflects our believes about reality. 

><u>Warning</u>: Although the alternative hypothesis contains the claim that we want to prove, it is important to note that in statistics we are not proving that the null hypothesis is true or false! We can only *reject* or *fail to reject* the null hypothesis based on our evidence in the data!!

**Question 5.2**
<br>{points: 1}

Suppose we want to test if the there exist a positive (linear) association between the response and the input variable. Which of the following alternative hypotheses is correct:

**A.** $H_1: \hat{\beta}_1 > 0 $

**B.** $H_1: \hat{\beta}_0 > 0 $

**C.** $H_1: \beta_1 \neq 0$ 

**D.** $H_1: \beta_1 > 0$ 

*Assign your answer to an object called `answer5.2`. Your answer should be one of `"A"`, `"B"`, `"C"`, or `"D"` surrounded by quotes.*

In [ ]:
# answer5.2 <- 

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_5.2()

### 5.3 The statistic and $p$-value

To test $H_0$ we can use our estimated slope and check how far the estimate $\hat{\beta}_1$ is from $0$. The test statistic is a relative measure that we can use based on our data. It can be found in the column `statistic` from `SLR_cancer_sample250_results`

Using classical results of the sampling distribution of the LS estimators, `lm` computes $p$-values for the test. In `SLR_cancer_sample250_results`, you can find the $p$-values in the column `p.value`. Note that, by default, the alternative hypothesis is $H_1: \beta_j \neq 0$, for all $j$th coefficients. However, we can change the default settings.

The `p.value` is interpreted as the probability, under $H_0$, that the test statistic is equal more extreme than the value observed in our sample (given in the column `statistic` of `SLR_cancer_sample250_results`). 

**Question 5.3**

Which of the following statement is correct??

A. The p-value is not the probability that the null hypothesis is true

B. The p-value is the probability that the alternative hypothesis is false

C. The p-value indicates the size or importance of the observed effect

D. The p-value is the probability that the observed effects were produced by random chance alone.

*Assign your answer to an object called `answer5.3`. Your answer should be one of `"A"`, `"B"`, `"C"`, or `"D"` surrounded by quotes.*

In [ ]:
# answer5.3 <- 

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_5.3()

### 5.4 Decision rule

The smaller the $p$-value, the stronger the evidence against $H_0$. Thus, small $p$-values (less than the significance level $\alpha$) indicate that the data provides enough statistical evidence against the null hypothesis of no association (i.e., to reject $H_0$).

> **Heads up**: in the last years, the scientific community has identified the "crisis of p-values". If you are interested in this topic you can read more about it in [this article](https://www.nature.com/articles/d41586-019-00857-9) and in the [ASA statement](https://www.stat.berkeley.edu/~aldous/Real_World/ASA_statement.pdf).

**Question 5.4**
<br>{points: 1}

Using the output stored in `SLR_cancer_sample250_results` and a significance level $\alpha = 0.05$, in plain words, what is the conclusion of the following hypothesis test?

$H_0: \beta_1 = 0 $

$H_1: \beta_1 \neq 0 $

A. We accept the alternative hypothesis; thus, the percentage of the county's populace in poverty has a statistically significant effect on the county's cancer mortality per capita (cases/100,000).

B. We reject the null hypothesis; thus, the percentage of the county's populace in poverty is statistically associated with the county's cancer mortality per capita (cases/100,000).

C. We fail to reject the null hypothesis; thus, the percentage of the county's populace in poverty is not statistically associated with the county's cancer mortality per capita (cases/100,000).

*Assign your answer to an object called `answer5.4`. Your answer should be one of `"A"`, `"B"`, or `"C"` surrounded by quotes.*

In [ ]:
# answer5.4 <- 

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_5.4()

### 6. Confidence Intervals

The values in `SLR_cancer_sample250_results` can also be used to compute confidence intervals for the regression parameters. 

> Remember that a 95% CI computed from the data is not a range of values that contain the true regression parameter with 95% probability. Once the interval has been computed based on the data, *nothing is random*! So it either covers or does not the true value. 

**Question 6.0**
<br>{points: 1}

Using `SLR_cancer_sample250` via `tidy()`, obtain the asymptotic 95% CIs for each regression parameters.

*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
# SLR_cancer_sample250_CIs <- 
#    ... %>%
#    ...(conf.int = ...) %>% 
#    mutate_if(is.numeric, round, 2)

# your code here
fail() # No Answer - remove if you provide an answer

SLR_cancer_sample250_CIs

In [ ]:
test_6.0()

### 7. The Sampling Distribution

We mentioned before that the estimators of the regression coefficient, $\hat{\beta}_0$ and $\hat{\beta}_1$, are *random variables*. Hence, they also have a *sampling distribution* (same as in STAT 201!). We need this distribution for confidence intervals and hypothesis testing.

The `lm` function will rely on the mathematical approximations. In this section, we will explore an alternative approach that you are familiar with: bootstrapping.  

> In Statistics, bootstrapping refers to sampling from our original sample with replacement to generate a *bootstrap sampling distribution*. The idea is to use the original sample as an *estimate* of the unknown population. 

Using *bootstrapping*, we generate a *long* list of estimates to *empirically* approximate the sampling distribution. The steps are as follows: 

- sample with replacement to obtain $B$ samples with size $n$;

- for each sample, compute the estimated regression coefficients;

- use the $B$ regression estimates of a given population parameter to calculate the sampling distribution of its estimator
(note: this list can also be used to estimate the mean and the standard error of the estimator).

> In regression, this is called *case* bootstrapping. While other bootstrap schemes exist; they are beyond the scope of this course.

#### 7.1 Bootstrap estimates

Let's generate a bootstrap list of estimates from `US_cancer_sample250`.

**Question 7.1.0**
<br>{points: 1}

Let's approximate the bootstrapping sampling distribution using `US_cancer_sample250`. 

Obtain $B = 1000$ sets of regression estimates by fitting a SLR $B$ times using their respective boostrap sample. Store the corresponding bootstrap estimates in the data frame `lm_boot250` of 1000 rows and two columns:

- `boot_intercept`: list of bootstrap intercepts
- `boot_slope`: list of bootstrap slopes 

*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
set.seed(123)  # DO NOT CHANGE!

# n <- ...
# B <- ...

# lm_boot250 <- replicate(..., {
#   sample_n(..., ..., ...) %>%
#     lm(..., data = .) %>%
#     .$coef
# })
# lm_boot250 <- data.frame(boot_intercept = lm_boot250[1, ], boot_slope = lm_boot250[2, ])

# your code here
fail() # No Answer - remove if you provide an answer

tail(lm_boot250)    

In [ ]:
test_7.1.0()

##### Using `infer`

The `infer` package can also help us generate bootstrap estimates of the regression coefficients. As an example, let's get the list of the slope of the SLR.

**Question 7.1.1**
<br>{points: 1}

Use the package `infer` to generate bootstrap samples from `US_cancer_sample250` and estimate the slope of the SLR for each bootstrap sample.

Obtain $B = 1000$ sets of regression estimates by fitting an SLR $B$ times using their respective bootstrap sample. Store the results in an object called `bootstrap_slope_infer`. 

The resulting object should have 1000 rows and two columns:

- `replicate`: a numeric ID to identify the bootstrap sample and estimate
- `stat`: the bootstrap slope estimated from the corresponding bootstrap sample

*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
set.seed(123)  # DO NOT CHANGE!

# bootstrap_slope_infer <- 
#   ... %>% 
#   specify(formula = ... ~ ...) %>%
#   generate(reps = ..., type = "bootstrap") %>% 
#   calculate(stat = "slope")

# your code here
fail() # No Answer - remove if you provide an answer

head(bootstrap_slope_infer)

Note: Each method uses different functions to generate bootstrap samples, so even if we use the same seeds, you don't get exactly the same samples and estimates.

In [ ]:
test_7.1.1()

#### 7.2 Bootstrap sampling distribution

Let's approximate the sampling distribution of the regression estimators in our problem using the list of bootstrapped estimates generated from `US_cancer_sample250`.

**Question 7.2.0**
<br>{points: 1}

Now that we have a list of bootstrapped estimates, we can visualize the bootstrap sampling distribution!

Let's focus on approximating the slope's sampling distribution. Use the list of bootstrapped estimates of the slopes stored in `lm_boot250` to plot the bootstrap sampling distribution of the least square estimator of the slope.

Name the `ggplot()` object as `slope_sampling_dist_250`.


*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
#slope_sampling_dist_250 <-  
#    lm_boot250 %>%
#    ggplot(..., aes(x = ...)) +
#    geom_histogram(bins = 30, color = "white", fill = "blue") +
#    xlab("...") +
#    ggtitle("Bootstrap sampling distribution for the estimator of the slope, n=250") +
#    coord_cartesian(xlim = c(0, 3))

# your code here
fail() # No Answer - remove if you provide an answer

slope_sampling_dist_250 

In [ ]:
test_7.2.0()

##### With `infer`

You can also use the package `infer` to visualize the distribution of the bootstrapped estimates generated. Check how similar both plots are!!

In [ ]:
#run this cell
bootstrap_slope_infer %>%
    visualize(bin=30)

#### Does the sample size matter?

Our estimator depends on a random sample and, thus, on its size! What happens to the estimates and the sampling distribution when we change the sample size?

Let's approximate the sampling distribution of estimators computed from samples of different sizes.

**Question 7.2.1**
<br>{points: 2}

We will now work with a sample of size $n=500$, stored in `US_cancer_sample500`.
Your job is to:

1. repeat the bootstrap experiment above using the `infer` package. 

2. plot the bootstrap sampling distribution

The `ggplot()` object's name will be `slope_sampling_dist_500`

*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
set.seed(123)  # DO NOT CHANGE!

US_cancer_sample500 <-     # DO NOT CHANGE!
    US_cancer_data %>%     # DO NOT CHANGE!
    slice_sample(n = 500)  # DO NOT CHANGE!

# B <- ...

# slope_sampling_dist_500 <- 
#    US_cancer_sample500 %>% 
#    ...(formula = ...) %>%
#    ...(reps = ..., type = ...) %>% 
#    ...(stat = ...) %>%
#    ...() + 
#    xlab("Estimated Slope") +
#    ggtitle("Bootstrap sampling distribution for the estimator of the slope, n=500") +
#    coord_cartesian(xlim = c(0, 3))



# your code here
fail() # No Answer - remove if you provide an answer

slope_sampling_dist_500 


In [ ]:
test_7.2.1()

**Question 7.2.2**
<br>{points: 1}

We will now work with a sample of size $n=1500$, stored in `US_cancer_sample1500`.
Once again, your job is to:

1. repeat the bootstrap experiment above using the *infer* package; 

2. plot the bootstrap sampling distribution

The `ggplot()` object's name will be `slope_sampling_dist_1500`

*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
set.seed(123)  # DO NOT CHANGE!

US_cancer_sample1500 <-     # DO NOT CHANGE!
    US_cancer_data %>%      # DO NOT CHANGE!
    slice_sample(n = 1500)  # DO NOT CHANGE!

# B <- ...

# slope_sampling_dist_1500 <- 
#    US_cancer_sample1500 %>% 
#    ...(formula = ...) %>%
#    ...(reps = ..., type = ...) %>% 
#    ...(stat = ...) %>%
#    ...() + 
#    xlab("Estimated Slope") +
#    ggtitle("Bootstrap sampling distribution for the estimator of the slope, n=1500")+
#    coord_cartesian(xlim = c(0, 3))



# your code here
fail() # No Answer - remove if you provide an answer

slope_sampling_dist_1500 

In [ ]:
test_7.2.2()

Looking at the 3 sampling distributions obtained by bootstrapping from samples of different sizes, side-by-side

In [ ]:
#run this cell

plot_grid(slope_sampling_dist_250 , slope_sampling_dist_500 ,slope_sampling_dist_1500, ncol=1 )

**Question 7.2.3**
<br>{points: 1}

Which of the following observations about the *bootstrap sampling distribution* is true?

A. The bootstrap sampling distribution of the estimator of the slope does not change with the size of the sample we bootstrapped from

B. The center of bootstrap sampling distribution of the estimator of the slope does not change with the size of the sample we bootstrapped from

C. The bootstrap sampling distribution of the estimator of the slope becomes tighter as the size of the sample we bootstrapped from increases


*Assign your answer to an object called `answer7.2.3`. Your answer should be one of `"A"`, `"B"`, or `"C"` surrounded by quotes.*

In [ ]:
# answer7.2.3 <- 

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_7.2.3()

#### 7.3 Bootstrap Confidence Intervals

In this exercise, we use the bootstrap sampling distribution to compute *bootstrap percentile* CIs of regression parameters.

This empirical sampling distribution can be used to make inferences, for example, to construct CIs (also done in STAT 201). We will use the percentile method to obtain CIs from the list of bootstrapped estimates.

**Question 7.3**
<br>{points: 1}

Using the $ B$ bootstrapped samples generated for a 250-sample size, stored in' lm_boot250', obtain a 95% confidence interval for the intercept and slope. The output must be a data frame with four columns: `statistic,` `avg,` `lower_ci,` and `upper_ci.`

*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
# boot_SLR_CIs <-
#     lm_boot250 %>%
#     pivot_longer(cols = contains('boot'), names_prefix = 'boot_', names_to = 'statistic') %>%
#     group_by(...) %>%
#     summarise(...)


# your code here
fail() # No Answer - remove if you provide an answer

boot_SLR_CIs

In [ ]:
test_7.3()

##### With `infer`

In [ ]:
#run this cell

percentile_ci <- 
    bootstrap_slope_infer %>% 
    get_confidence_interval(type = "percentile", level = 0.95)

percentile_ci

In [ ]:
#run this cell

bootstrap_slope_infer %>%
    visualize(bin=20) + 
    shade_confidence_interval(endpoints = percentile_ci) 